# Import Library

In [1]:
import gmaps
import pandas as pd
import numpy as np
import requests
import json
from config import gkey
# Configure gmaps
gmaps.configure(api_key=gkey)

# Read csv file

In [2]:
# Create a dataframe
weather_df = pd.read_csv('../WeatherPy/weatherpy_data.csv')
weather_df.head()

,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,ostrovnoy,ru,2020-03-13 19:35:15,68.05,39.51,22.53,95.0,100.0,23.94
1,poum,nc,2020-03-13 19:35:15,-20.23,164.02,81.36,85.0,100.0,13.82
2,vaini,to,2020-03-13 19:33:36,-21.20,-175.20,77.00,94.0,75.0,12.75
3,saint-philippe,re,2020-03-13 19:33:10,-21.36,55.77,77.00,88.0,20.0,16.11
4,conde,br,2020-03-13 19:35:16,-7.26,-34.91,78.80,94.0,40.0,4.70


## Create a heat map that displays the humidity for every city 

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]
Humidity = weather_df["Humidity (%)"].astype(float)
maxhumidity = Humidity.max()

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity,
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Find ideal weather condition

### Narrow down the DataFrame to find an ideal weather condition.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.

In [5]:
new_weather_df = weather_df.loc[(weather_df["Max Temperature (F)"] > 70) & 
                                (weather_df["Max Temperature (F)"] < 80) & 
                                (weather_df["Wind Speed (mph)"]<10) &
                                (weather_df["Cloudiness (%)"] == 0) , :]

new_weather_df.reset_index(inplace=True)
del new_weather_df['index']
print(f"There are {len(new_weather_df)} cities satisfying the ideal weather condition.")
new_weather_df

There are 9 cities satisfying the ideal weather condition.


,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,araouane,ml,2020-03-13 19:35:25,18.90,-3.53,75.22,10.0,0.0,5.95
1,narrabri,au,2020-03-13 19:35:33,-30.32,149.78,75.20,50.0,0.0,4.70
2,arraial do cabo,br,2020-03-13 19:35:59,-22.97,-42.03,77.00,94.0,0.0,5.82
3,acajutla,sv,2020-03-13 19:33:23,13.59,-89.83,73.99,93.0,0.0,1.99
4,dakar,sn,2020-03-13 19:32:50,14.69,-17.44,78.80,73.0,0.0,4.70
5,lazaro cardenas,mx,2020-03-13 19:36:17,17.96,-102.20,79.93,71.0,0.0,7.23
6,matara,lk,2020-03-13 19:36:23,5.95,80.54,71.44,90.0,0.0,6.67
7,roma,au,2020-03-13 19:36:05,-26.58,148.78,74.19,49.0,0.0,9.95
8,cangucu,br,2020-03-13 19:36:35,-31.39,-52.68,73.99,87.0,0.0,1.01


## Build a Hotel Map

In [6]:
hotels = []

for i in range(len(new_weather_df)):
    lat = new_weather_df.loc[i]['Latitude']
    lng = new_weather_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        print(f"Processing Record {i}, City Name: {new_weather_df.loc[i]['City']}.")
        hotels.append(response['results'][0]['name'])
        
    except:
        hotels.append(np.nan)
        print("Oops, Nearest hotel not found!")
        
print("-----------------------------")        
print("Data Retrieval Complete")
print("-----------------------------")

Processing Record 0, City Name: araouane.
Processing Record 1, City Name: narrabri.
Processing Record 2, City Name: arraial do cabo.
Processing Record 3, City Name: acajutla.
Processing Record 4, City Name: dakar.
Processing Record 5, City Name: lazaro cardenas.
Processing Record 6, City Name: matara.
Processing Record 7, City Name: roma.
Processing Record 8, City Name: cangucu.
-----------------------------
Data Retrieval Complete
-----------------------------


In [7]:
weather_data=new_weather_df.copy()
weather_data["Hotel Name"] = hotels
weather_data=weather_data.dropna()
weather_data

,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,araouane,ml,2020-03-13 19:35:25,18.90,-3.53,75.22,10.0,0.0,5.95,Araouane
1,narrabri,au,2020-03-13 19:35:33,-30.32,149.78,75.20,50.0,0.0,4.70,Narrabri
2,arraial do cabo,br,2020-03-13 19:35:59,-22.97,-42.03,77.00,94.0,0.0,5.82,Cabo Frio
3,acajutla,sv,2020-03-13 19:33:23,13.59,-89.83,73.99,93.0,0.0,1.99,Acajutla
4,dakar,sn,2020-03-13 19:32:50,14.69,-17.44,78.80,73.0,0.0,4.70,Dakar
5,lazaro cardenas,mx,2020-03-13 19:36:17,17.96,-102.20,79.93,71.0,0.0,7.23,Lázaro Cárdenas
6,matara,lk,2020-03-13 19:36:23,5.95,80.54,71.44,90.0,0.0,6.67,Matara
7,roma,au,2020-03-13 19:36:05,-26.58,148.78,74.19,49.0,0.0,9.95,Roma
8,cangucu,br,2020-03-13 19:36:35,-31.39,-52.68,73.99,87.0,0.0,1.01,Canguçu


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in weather_data.iterrows()]
locations = weather_data[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))